In [1]:
import pandas as pd
import os

In [2]:
snp = "190722_Clusters_Almeria.cohort.snp.hf.vcf"
indel = "190722_Clusters_Almeria.cohort.indel.hf.vcf"

In [13]:
def extract_reference_vcf(input_vcf):
    """
    Read file until header ends and pick the first field corresponding to reference
    """
    with open(input_vcf, "r") as f:
        next_line = f.readline().strip()
        while next_line.startswith("#"):
            #print(next_line)
            next_line = f.readline()
        
    reference = next_line.split()[0]
    return reference

In [14]:
extract_reference_vcf(snp)

'MTB_anc'

In [14]:
indel[::-1]

'fcv.fh.ledni.trohoc.airemlA_sretsulC_227091'

In [15]:
def longest_common_suffix(list_of_strings):
    """
    Return the longest common suffix in a list of strings
    Adapted from https://gist.github.com/willwest/ca5d050fdf15232a9e67
    """
    reversed_strings = [s[::-1] for s in list_of_strings]
    reversed_lcs = os.path.commonprefix(reversed_strings)
    lcs = reversed_lcs[::-1]
    return lcs

In [16]:
def combine_vcf(vcf_file_1, vcf_file_2, name_out=False):
    """
    Merge vcf files by position (POS) and ALT variants
    """ 
    input_vcf_1 = os.path.abspath(vcf_file_1)
    input_vcf_2 = os.path.abspath(vcf_file_2)
    
    input_vcf_dir_name = os.path.dirname(vcf_file_1)
    
    if name_out == False:
        prefix = os.path.commonprefix([input_vcf_1, input_vcf_2])
        suffix = longest_common_suffix([input_vcf_1, input_vcf_2])
        output_file = prefix + "combined" + suffix
        
    else:
        output_file = os.path.abspath(name_out)
    
    header_lines_list = []
    header_lines_list_f1 = []
    #Extract filter info from header since is the only difference between headers
    with open(input_vcf_2, "r") as f2:
            for line in f2:
                if line.startswith("#"):
                    header_lines_list.append(line)
                    
    #Extract filter info from file1
    with open(input_vcf_1, "r") as f1:
        for line in f1:
            if line.startswith("##FILTER") and line not in header_lines_list:
                header_lines_list_f1.append(line)
                
    #Combine header info, addiing filter info together
    #Extract all lines starting with ##FILTER
    filter_fields = [i for i in header_lines_list if i.startswith('##FILTER')]
    #Obtain the index of the first line with index
    filter_index = header_lines_list.index(filter_fields[0])
    #Include the list within the header
    header_lines_list[filter_index:filter_index] = header_lines_list_f1

    variant_lines = []
    
    with open(input_vcf_1, "r") as f1:
        with open(input_vcf_2, "r") as f2:
            with open(output_file, "w+") as fout:
                fout.write("".join(header_lines_list))
                for line in f1:
                    if not line.startswith("#"):
                        variant_lines.append(line)
                for line in f2:
                    if not line.startswith("#") and line not in variant_lines:
                        variant_lines.append(line)
                fout.write("".join(variant_lines))

In [ ]:
def combine_vcf_legacy(vcf_file_1, vcf_file_2, name_out=False):
    """
    Merge vcf files by position (POS) and ALT variants
    """ 
    input_vcf_1 = os.path.abspath(vcf_file_1)
    input_vcf_2 = os.path.abspath(vcf_file_2)
    #input_vcf_dir_name = os.path.dirname(vcf_file_1)
    
    if name_out == False:
        prefix = os.path.commonprefix([input_vcf_1, input_vcf_2])
        suffix = longest_common_suffix([input_vcf_1, input_vcf_2])
        output_file = prefix + "combined" + suffix
        
    else:
        output_file = os.path.abspath(name_out)
    
    list_pos = []
    
    with open(input_vcf_1, "r") as f1:
        with open(input_vcf_2, "r") as f2:
            with open(output_file, "w+") as fout:
                for line in f1:
                    if line.startswith("#"): #write header only from first vcf
                        fout.write(line)
                    else:
                        position =  int(line.split("\t")[1])
                        if position not in list_pos:
                            fout.write(line)
                            list_pos.append(position)
                for line in f2:
                    if line.startswith("#"):
                        f2.readline()
                    else:
                        position =  int(line.split("\t")[1])
                        if position not in list_pos:
                            fout.write(line)
                            list_pos.append(position)


In [17]:
%%time
combine_vcf(snp, indel)

CPU times: user 2.33 s, sys: 49.7 ms, total: 2.38 s
Wall time: 2.37 s


In [80]:
test=["rara", "lol", "foo", "raro"]

In [81]:
#test.insert(2, ",".join(['xxx', 'yyy']))
test[2:2]=['xxx', 'yyy']

In [82]:
test

['rara', 'lol', 'xxx', 'yyy', 'foo', 'raro']

In [50]:
result = [i for i in test if i.startswith('ra')]

In [51]:
result

['rara', 'raro']

In [54]:
test.index(result[0])

0

In [ ]:
#https://pythonforbiologists.com/randomly-sampling-reads-from-a-fastq-file
"""
record_number = 0
with open("test.fastq") as input:
    with open("sample.fastq", "w") as output:
        for line1 in input:
            line2 = input.next()
            line3 = input.next()
            line4 = input.next()
            if record_number % 10 == 0:
                    output.write(line1)
                    output.write(line2)
                    output.write(line3)
                    output.write(line4)
            record_number += 1
"""

In [18]:
def combine_vcf(vcf_file_1, vcf_file_2, name_out=False):
    """
    Merge vcf files by position (POS) and ALT variants
    """ 
    input_vcf_1 = os.path.abspath(vcf_file_1)
    input_vcf_2 = os.path.abspath(vcf_file_2)
    
    input_vcf_dir_name = os.path.dirname(vcf_file_1)
    
    if name_out == False:
        prefix = os.path.commonprefix([input_vcf_1, input_vcf_2])
        suffix = longest_common_suffix([input_vcf_1, input_vcf_2])
        output_file = prefix + "combined" + suffix
        
    else:
        output_file = os.path.abspath(name_out)
    
    list_pos = []
    filter_lines_f1 = ""
    filter_lines_f2_list = []
    #Extract filter info from header since is the only difference between headers
    with open(input_vcf_2, "r") as f2:
            for line in f2:
                if line.startswith("##FILTER"):
                    filter_lines_f2_list.append(line)
                    
    #Handle combined header into variable
    
    with open(input_vcf_1, "r") as f1:
        for line in f1:
            if line.startswith("##FILTER") and line not in filter_lines_f2_list:
                filter_lines_f1 = filter_lines_f1 + line
                filter_lines_f2_list.append(line)
    print(line)


    with open(input_vcf_1, "r") as f1:
        with open(input_vcf_2, "r") as f2:
            with open(output_file, "w+") as fout:
                for line in f1:
                    line2 = f1.readline()
                    if line.startswith("##FILTER") and not line2.startswith("##FILTER"):
                        fout.write(line)
                        fout.write(filter_lines_f1)
                        fout.write(line2)
                        #print(line, line2)
                    elif line.startswith("#") and line2.startswith("#"):
                        fout.write(line)
                        fout.write(line2)
                        #print(line, line2)
                    else:
                        position = int(line.split("\t")[1])
                        position2 = int(line2.split("\t")[1])
                        if position not in list_pos:
                            fout.write(line)
                            fout.write(line2)
                            list_pos.append(position)
                            list_pos.append(position2)
                for line in f2:
                    if line.startswith("#"):
                        f2.readline()
                    else:
                        position =  int(line.split("\t")[1])
                        if position not in list_pos:
                            fout.write(line)
                            list_pos.append(position)
    print(len(list_pos))